In [1]:
from __future__ import annotations

import attrs

import vocalpy as voc

In [ ]:
@attrs.define
class SequenceDataset:
    sequences: list[vocalpy.Sequence] = attrs.field()

    def to_sqlite(db_path: str):
        # TODO: write this so that we always re-create the whole thing from scratch;
        # don't bother checking whether anything is in the database because we *know* we just made it
        # that's the whole point
        pass

    def from_sqlite(db_path: str):
        pass

In [2]:
cd /Users/davidnicholson/Documents/repos/coding/vocalpy/vocalpy

/Users/davidnicholson/Documents/repos/coding/vocalpy/vocalpy


In [4]:
ls tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/

032312/ 032412/


In [5]:
data_dir = 'tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/'

In [6]:
cbins = voc.paths.from_dir(data_dir, 'cbin')

In [7]:
aud = voc.Audio(path=cbins[0])

In [8]:
audios = [voc.Audio.read(cbin) for cbin in cbins]

In [9]:
segment_params = {
    'threshold': 1500,
    'min_dur': 0.01,
    'min_silent_dur': 0.006,
}
segmenter = voc.Segmenter(segment_params=segment_params)

In [10]:
seqs = segmenter.segment(audios)

[                                        ] | 0% Completed | 122.28 us

/Users/davidnicholson/Documents/repos/coding/vocalpy/vocalpy/src/vocalpy/signal/segment.py:56: UserWarning: Values in `data` would overflow when squaring because of dtype, >i2;casting to a larger integer dtype to avoid
  smoothed = smooth(data, samplerate, smooth_win)


[########################################] | 100% Completed | 105.22 ms


In [11]:
import sqlalchemy

In [12]:
import enum

from sqlalchemy import (
    Column,
    create_engine,
    Enum,
    Float,
    ForeignKey,
    Integer,
    String,
    Table,
)
from sqlalchemy.orm import DeclarativeBase

from sqlalchemy.orm import relationship

import vocalpy as voc

In [13]:
class Base(DeclarativeBase):
    pass

In [14]:
class Audio(Base):
    __tablename__ = 'audios'
    
    id = Column(Integer, primary_key=True)
    path = Column(String)
    
    sequences = relationship("Sequence", back_populates="audio")

    def __repr__(self):
        return f"Audio(id={self.id!r}, path={self.path!r})"

In [15]:
class SegmentParams(Base):
    __tablename__ = 'segment_params'
    id = Column(Integer, primary_key=True)
    path = Column(String)
    
    sequences = relationship("Sequence", back_populates="segment_params")

    def __repr__(self):
        return f"SegmentParams(id={self.id!r}, path={self.path!r})"

In [16]:
class Sequence(Base):
    __tablename__ = 'sequences'
    
    id = Column(Integer, primary_key=True)
    audio_id = Column(Integer, ForeignKey('audios.id'))
    onset = Column(Float)
    offset = Column(Float)
    method = Column(String)  # should this be a table?
    segment_params_id = Column(Integer, ForeignKey('segment_params.id'))

    audio = relationship("Audio", back_populates="sequences")
    segment_params = relationship("SegmentParams", back_populates="sequences")
    
    units = relationship("Unit", back_populates="sequence")

    def __repr__(self):
        return (f"Sequence(id={self.id!r}, audio_id={self.audio_id!r}, onset={self.onset!r}, offset={self.offset!r}, method={self.method!r}, "
                f"segment_params_id={self.segment_params_id!r}, audio={self.audio!r})")

In [17]:
class Unit(Base):
    __tablename__ = 'units'
    
    id = Column(Integer, primary_key=True)
    sequence_id = Column(Integer, ForeignKey('sequences.id'))
    onset = Column(Float)
    offset = Column(Float)
    
    sequence = relationship("Sequence", back_populates="units")
    
    def __repr__(self):
        return f"Unit(id={self.id!r}, sequence_id={self.sequence_id!r}, onset={self.onset!r}, offset={self.offset!r})"

In [18]:
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session

In [19]:
def get_sqlite_engine(db_path=':memory:', echo=True):
     return create_engine(f'sqlite:///{db_path}', echo=echo)

def get_session(engine):
    Session = sessionmaker(bind=engine)
    session = Session()
    return session

In [20]:
import json
import pathlib

def to_sqlite(seqs, db_name=':memory:', dst=None, echo=True):
    if dst:
        dst = pathlib.Path(dst)
        if not dst.exist() or not dst.is_dir():
            raise NotADirectory(
                f"`dst` not found or not recognized as a directory: {dst}"
            )
        url = f'sqlite:///{dst}{db_name}'
    else:
        url = f'sqlite:///{db_name}'

    engine = create_engine(url, echo=echo)

    Base.metadata.create_all(engine)

    # TODO: handle checking here -- 
    # actually make sure in Segmenter that `segment_params` can be serialized
    
    # make a set to get unique values, 
    # then make a list so we can do uniq_segment_params.index in loop below

    segment_params_map = {}
    uniq_segment_params = []
    # need to do it this way since we can't hash a dict to use `set` to find unique
    # https://stackoverflow.com/questions/11092511/list-of-unique-dictionaries
    for seq in seqs:
        if seq.segment_params not in uniq_segment_params:
            uniq_segment_params.append(segment_params)

    orm_segment_params = []
    for ind, segment_params_dict in enumerate(uniq_segment_params):
        fname = f'segment-params-{ind + 1}.json'
        if dst:
            segment_params_json_path = dst / fnane
        else:
            segment_params_json_path = pathlib.Path(fname)
        with segment_params_json_path.open('w') as fp:
            json.dump(segment_params_dict, fp)
        orm_segment_params.append(
            SegmentParams(path=str(segment_params_json_path))
        )

    with Session(engine) as session, session.begin():
        for an_orm_segment_params in orm_segment_params:
            session.add(an_orm_segment_params)
        
        for seq in seqs:
            audio = Audio(path=str(seq.audio.path))
            session.add(audio)

            # make and add sequence, referring to audio and segment params
            ind = uniq_segment_params.index(seq.segment_params)
            an_orm_segment_params = orm_segment_params[ind]

            sequence = Sequence(
                audio=audio,
                segment_params=an_orm_segment_params,
                onset=seq.onset,
                offset=seq.offset,
                method=seq.method,
            )
            session.add(sequence)

            # make and add units
            for seq_unit in seq.units:
                unit = Unit(
                    onset=seq_unit.onset,
                    offset=seq_unit.offset,
                    sequence=sequence
                )
                session.add(unit)
        # ---- implicit session.commit() when we __exit__ context + begin() from above

In [21]:
url = f'sqlite:///test.db'

engine = create_engine(url, echo=True)

Base.metadata.create_all(engine)

2023-05-08 08:45:46,958 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-08 08:45:46,959 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("audios")
2023-05-08 08:45:46,961 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-08 08:45:46,963 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("segment_params")
2023-05-08 08:45:46,964 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-08 08:45:46,966 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sequences")
2023-05-08 08:45:46,967 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-08 08:45:46,970 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("units")
2023-05-08 08:45:46,972 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-08 08:45:46,974 INFO sqlalchemy.engine.Engine COMMIT


In [22]:
with Session(engine) as session, session.begin():
    stmt = select(Sequence)
    this = session.scalars(stmt).all()
    print(this[0])

2023-05-08 08:45:47,387 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-08 08:45:47,406 INFO sqlalchemy.engine.Engine SELECT sequences.id, sequences.audio_id, sequences.onset, sequences."offset", sequences.method, sequences.segment_params_id 
FROM sequences
2023-05-08 08:45:47,408 INFO sqlalchemy.engine.Engine [generated in 0.00132s] ()
2023-05-08 08:45:47,414 INFO sqlalchemy.engine.Engine SELECT audios.id AS audios_id, audios.path AS audios_path 
FROM audios 
WHERE audios.id = ?
2023-05-08 08:45:47,415 INFO sqlalchemy.engine.Engine [generated in 0.00125s] (1,)
Sequence(id=1, audio_id=1, onset=0.0, offset=12.30528125, method='audio_amplitude', segment_params_id=1, audio=Audio(id=1, path='tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0808.138.cbin'))
2023-05-08 08:45:47,417 INFO sqlalchemy.engine.Engine COMMIT
